In [1]:
import numpy as np
import pandas as pd
import csv
import os
from SepsisCheck import sepsischeck_utilities_for_pkl as su
from sklearn.metrics import precision_recall_fscore_support as score
#from sklearn.metrics import average_precision_score as score
from sklearn.metrics import classification_report as report
from sklearn.metrics import roc_auc_score as auroc
from sklearn.metrics import confusion_matrix

### Take a look at the results of sepsis check on patients

In [2]:
def convert_result_to_df(filename):
    result_dict = []
    with open(filename, 'r') as f:
        for i, line in enumerate(f):
            line_list = line.split()
            line_list = [s.strip(',') for s in line_list]
            line_list = [s.replace(',', '') for s in line_list]
            result_dict.append(dict({'Subject ID': line_list[2], 
                                        'Hadm_ID': line_list[4],
                                        'ts_ind': line_list[6],
                                        'Sepsis': line_list[8], 
                                        't_sepsis': line_list[10],
                                        't_sofa': line_list[12], 
                                        't_cultures': line_list[14], 
                                        't_IV': line_list[16], 
                                        't_sus': line_list[18]}))
    df = pd.DataFrame.from_records(result_dict)

    return df

In [18]:
# load data we classified
path = "../data/exp1/forecasting_exp1/mimic_iii_preprocessed_forecasting1.pkl"
preds = "../data/exp1/forecasting_preds/forecasting_preds_test/content/forecasting_preds_test.pkl"
data = pd.read_pickle(path)
preds = pd.read_pickle(preds)
#sort by ts_ind as that is how the results are sorted
data1 = preds[0].sort_values(by=["ts_ind"]).drop_duplicates("ts_ind")
IDs = list(data1["ts_ind"].unique())
#make ground truth for scoring, reset index after sorting. Index 0 -> ts_ind 0
ground_truth = data1[["sepsis_label", "ts_ind"]].reset_index(drop=True) #10498 labels sorted by ts_ind from low to high

#load positive and negative with IV and cultures feature from file, more than 10498 probably -> faster than computing again 
# -> ToDo: Keep only IDs that are in data1 -> smaller than 10498
 
with open("./features/possible_predicitons/forecast_possible_pos_predictions.csv") as f:
    reader = csv.reader(f)
    possible_pos_ = [int(row[0]) for row in reader]
with open("./features/possible_predicitons/forecast_possible_neg_predictions.csv") as f:
    reader = csv.reader(f)
    possible_neg_ = [int(row[0]) for row in reader]

# lists of ts_ind where each item was forecast, but also contained both IV and cultures originally
possible_pos_predictions = list(set(IDs).intersection(set(possible_pos_)))
possible_neg_predictions = list(set(IDs).intersection(set(possible_neg_)))

#indeces of all patients with IV and cultures
all_possible_ = possible_pos_predictions + possible_neg_predictions
all_possible_.sort()

#get labels for all patients that have IV and cultures
adj_ground_truth = ground_truth.loc[ground_truth["ts_ind"].isin(all_possible_)]

ground_truth = ground_truth.sort_values(by=["ts_ind"])
adj_ground_truth = adj_ground_truth.sort_values(by=["ts_ind"])

# load times of sepsis from file
times = pd.read_csv("./features/possible_predicitons/times_cleaned.csv")
t_i = pd.read_csv("./features/possible_predicitons/ts_times_cleaned.csv")


# df for holding results
col = ["experiment", "t_sepsis_mean", "24_hour_window", "t_ident", "AUROC", "AUROC_adj","precision_raw", "precision_adj", "recall_raw", "recall_adj", "f1_raw", "f1_adj", "support", "support_adj", "cm", "cm_adj"]
df = pd.DataFrame(columns=col)

In [27]:
# experimental and not really tested, tries to find how close SepsisCheck's time of sepsis is to actual time of sepsis reported in clinical notes.
def compare_t_sepsis(result_file, time=times, IDs=t_i):
    count = []
    for index, row in time.iterrows():
        if int(row["diffs"]) > -1:
            sub_id = int(row["SUBJECT_ID"])
            dif = float(row["diffs"])
            p = result_file["t_sepsis"].loc[result_file["Subject ID"] == str(sub_id)]
            for i, sep in enumerate(p):
                try:
                    count.append(abs(dif - float(sep)))
                except:
                    continue
    t = pd.DataFrame(count, columns=["diffs"])

    h = 0
    c = 0
    for index, x in enumerate(IDs):
        try:
            sub_id = int(IDs.at[index, "SUBJECT_ID"])
            ts_inds = result_file["ts_ind"].loc[result_file["Subject ID"] == str(sub_id)]
        except:
            h += 1
            continue
        
        for ind in ts_inds:
            p = result_file["t_sepsis"].loc[result_file["ts_ind"] == str(ind)]
            pred = float(p)
            dif = IDs.at[index, "diffs"]
            try:
                difference = dif - pred
                if difference <= 24:
                    c += 1
            except:
                h += 1
                continue 




    return (t["diffs"].mean()), (c / (len(IDs) - h))

def get_t_identification(sofa, IV, cult):
    values =  []
    # for each triplet, get the max -> how much time needs to be observed for a identification
    for i in range(len(sofa)):
        values.append(max(sofa[i], IV[i], cult[i]))
    return sum(values) / len(values)

In [21]:
def compute_results(path):
    # load results and get predicted arrays

    df = convert_result_to_df(path)
    # get mean t_sepsis to compare methods
    df["t_sepsis"] = df["t_sepsis"].replace({"False":np.nan})
    t_sep, hour_window = compare_t_sepsis(df)

    df["ts_ind"] = df["ts_ind"].astype(int)
    df = df.sort_values(by=["ts_ind"], ascending=True)
    # get indexes of False and True Sepsis labels in results
    noSeps = df.loc[df["Sepsis"] == "False"]
    Seps = df.loc[df["Sepsis"] == "True"]
    neg_hadm_IDs = list(map(int, noSeps["ts_ind"]))
    pos_hadm_IDs = list(map(int, Seps["ts_ind"]))

    # get t of identification of positive labeled patients
    t_ident = get_t_identification(Seps["t_sofa"].astype(float).values, Seps["t_IV"].astype(float).values, Seps["t_cultures"].astype(float).values)
    
    #make predicted df for scoring
    predicted = pd.DataFrame(index=df["ts_ind"], columns=["pred"])
    # 0 for each patient where Sepsis was False
    predicted[predicted.index.isin(neg_hadm_IDs)] = 0
    # 1 for each patient where Sepsis was True
    predicted[predicted.index.isin(pos_hadm_IDs)] = 1
    adj_predicted = predicted.loc[predicted.index.isin(all_possible_)]

    

    # precision, recall f1 (fbeta=1.0) on raw data
    precision, recall, f1_score, support = score(y_true=ground_truth["sepsis_label"].values.astype(int), y_pred=predicted["pred"].values.astype(int), average="weighted")
    auroc_score = auroc(y_true=ground_truth["sepsis_label"].values.astype(int), y_score=predicted["pred"].values.astype(int), average="weighted")
    cm=confusion_matrix(ground_truth["sepsis_label"].values.astype(int), predicted["pred"].values.astype(int)).ravel()
    #print(report(y_true=ground_truth.values.astype(int), y_pred=predicted["pred"].values.astype(int)))

    # precision, recall f1 (fbeta=1.0) on patients that include IV and cultures
    precision_adj, recall_adj, f1_score_adj, support_adj = score(y_true=adj_ground_truth["sepsis_label"].values.astype(int), y_pred=adj_predicted["pred"].values.astype(int), average="weighted")
    auroc_score_adj = auroc(y_true=adj_ground_truth["sepsis_label"].values.astype(int), y_score=adj_predicted["pred"].values.astype(int), average="weighted")
    cm_adj=confusion_matrix(adj_ground_truth["sepsis_label"].values.astype(int), adj_predicted["pred"].values.astype(int)).ravel()

    #print(report(y_true=adj_ground_truth.values.astype(int), y_pred=adj_predicted["pred"].values.astype(int)))

    return  t_sep, hour_window, t_ident, auroc_score, auroc_score_adj, precision, recall, f1_score, support, precision_adj, recall_adj, f1_score_adj, support_adj, cm, cm_adj

In [22]:
def cleanFilename(sourcestring,  removestring =" %:/,.\\[]<>*?"):
    #remove the undesireable characters
    return ''.join([c for c in sourcestring if c not in removestring])



Get results only on observed data, full data and cutoff at 120 hours.
Only need to run if there is no '/results/forecast/sepsis-3_forecast_report.csv'

In [ ]:
results_path = "./results/forecast/test/sepsis-3/"
directory = os.fsencode(results_path)
for file in os.listdir(directory):
    name = file.decode('UTF-8')
    t_sepsis, hour_window, t_ident, auroc_score, auroc_score_adj, precision, recall, f1_score, support, precision_adj, recall_adj, f1_score_adj, support_adj, cm, cm_adj = compute_results(os.path.join(directory, file))
    new_row = {"experiment":name[:-4],"t_sepsis_mean":t_sepsis, "24_hour_window":hour_window, "t_ident":t_ident, "AUROC":auroc_score,"AUROC_adj":auroc_score_adj,"precision_raw":precision, "precision_adj":precision_adj, "recall_raw":recall, "recall_adj":recall_adj, "f1_raw":f1_score, "f1_adj":f1_score_adj, "support":support, "support_adj":support_adj, "cm": cm, "cm_adj":cm_adj}
    df2 = df.append(new_row, ignore_index=True)
    df2.to_csv('./results/forecast/sepsis-3_forecast_report.csv', mode='a', index=False, header=False)

### Results only on observed data
no forecast, sorted by f1_raw. Looking at the True Positives in the "cm" column, we can deduct that the (very close) scores are very heavily dependent on true negatives. Even though the last two rows have only 39 True Positives the F1 score is only 1% worse.

In [26]:
# cm = confusion matrix: tn, fp, fn, tp
results_3 = pd.read_csv("results/forecast/sepsis-3_forecast_report.csv", names=col, header=None)
y = results_3[["experiment", "f1_raw", "cm"]].sort_values(["f1_raw"], ascending=False)
y

,experiment,f1_raw,cm
7,sepsis-3_no-prediction_24-12_240-240,0.876760,[8240 518 643 315]
2,cutoffsepsis-3_no-prediction_24-12_240-240,0.875187,[8278 480 674 284]
3,cutoffsepsis-3_no-prediction_24-12_48-72,0.874505,[8335 423 707 251]
1,cutoffsepsis-3_no-prediction_24-12_24-96,0.873090,[8351 407 724 234]
4,cutoffsepsis-3_no-prediction_48-24_168-168,0.871523,[8193 565 656 302]
8,sepsis-3_no-prediction_48-24_168-168,0.871408,[8154 604 636 322]
0,cutoffsepsis-3_no-prediction_24-12_12-24,0.871317,[8476 282 790 168]
9,sepsis-3_no-prediction_48-24_24-72,0.870590,[8293 465 713 245]
5,cutoffsepsis-3_no-prediction_48-24_24-72,0.870441,[8293 465 714 244]
6,cutoffsepsis-3_no-prediction_6-3_1-3,0.861295,[8709 49 919 39]


In [ ]:
results_path = "./results/forecast/test/sepsis-3_on_forecast_data"
directory = os.fsencode(results_path)
for file in os.listdir(directory):
    name = file.decode('UTF-8')
    t_sepsis, hour_window, t_ident, auroc_score, auroc_score_adj, precision, recall, f1_score, support, precision_adj, recall_adj, f1_score_adj, support_adj, cm, cm_adj = compute_results(os.path.join(directory, file))
    new_row = {"experiment":name[:-4],"t_sepsis_mean":t_sepsis, "24_hour_window":hour_window, "t_ident":t_ident, "AUROC":auroc_score,"AUROC_adj":auroc_score_adj,"precision_raw":precision, "precision_adj":precision_adj, "recall_raw":recall, "recall_adj":recall_adj, "f1_raw":f1_score, "f1_adj":f1_score_adj, "support":support, "support_adj":support_adj, "cm": cm, "cm_adj":cm_adj}
    df2 = df.append(new_row, ignore_index=True)
    df2.to_csv('./results/forecast/on_forecast_report.csv', mode='a', index=False, header=False)

### Results on observed data + 1 hour forecast data
-> Still missing: Only on forecast data
Different thresholds yield different results, note that the difference between TH1.0025 (2.007817025) and TH1.00275 (2.0083177275) is extremely small, which is why finding a good threshold is so difficult; base is 2.002810 (for Blood Culture), and was found through clustering, and Jing Hua's analysis defined the search direction, which was very useful. 

In [29]:
# tn, fp, fn, tp
results_4 = pd.read_csv("./results/forecast/on_forecast_report.csv", names=col, header=None)
z = results_4[["experiment", "f1_raw", "cm"]].sort_values(["f1_raw"], ascending=False)
z

,experiment,f1_raw,cm
5,TH1.0025_sepsis-3_prediction_24-12_48-72,0.873794,[8255 503 672 286]
3,TH1.0025_sepsis-3_prediction_24-12_24-96,0.873677,[8295 463 693 265]
4,TH1.0025_sepsis-3_prediction_24-12_240-240,0.873287,[8189 569 641 317]
8,TH1.00275_sepsis-3_prediction_24-12_240-240,0.872978,[8196 562 647 311]
1,TH1.00225_sepsis-3_prediction_24-12_240-240,0.872908,[8169 589 633 325]
10,TH1.002_sepsis-3_prediction_24-12_240-240,0.872316,[8139 619 621 337]
9,TH1.002_sepsis-3_prediction_24-12_12-24,0.871763,[8444 314 774 184]
2,TH1.0025_sepsis-3_prediction_24-12_12-24,0.871553,[8460 298 782 176]
11,TH1.003_sepsis-3_prediction_24-12_12-24,0.870751,[8463 295 788 170]
7,TH1.0025_sepsis-3_prediction_48-24_24-72,0.869695,[8214 544 680 278]


In [ ]:
results_path = "./results/forecast/test/compare/"
directory = os.fsencode(results_path)
for file in os.listdir(directory):
    name = file.decode('UTF-8')
    t_sepsis, hour_window, t_ident, auroc_score, auroc_score_adj, precision, recall, f1_score, support, precision_adj, recall_adj, f1_score_adj, support_adj, cm, cm_adj = compute_results(os.path.join(directory, file))
    new_row = {"experiment":name[:-4],"t_sepsis_mean":t_sepsis, "24_hour_window":hour_window, "t_ident":t_ident, "AUROC":auroc_score,"AUROC_adj":auroc_score_adj,"precision_raw":precision, "precision_adj":precision_adj, "recall_raw":recall, "recall_adj":recall_adj, "f1_raw":f1_score, "f1_adj":f1_score_adj, "support":support, "support_adj":support_adj, "cm": cm, "cm_adj":cm_adj}
    df2 = df.append(new_row, ignore_index=True)
    df2.to_csv('./results/forecast/full_forecast_report.csv', mode='a', index=False, header=False)

### comparison
for ease of reading, only TH1.0025 and cutoff is included

For each pair of TH1.0025 and cutoffsepsis-3, the TH1.0025 version got more True Positives.

In [2]:
# tn, fp, fn, tp
col = ["experiment", "t_sepsis_mean", "24_hour_window", "t_ident","AUROC", "AUROC_adj","precision_raw", "precision_adj", "recall_raw", "recall_adj", "f1_raw", "f1_adj", "support", "support_adj", "cm", "cm_adj"]
results = pd.read_csv("./results/forecast/full_forecast_report.csv", names=col, header=None)
x = results[["experiment", "f1_raw", "cm"]].sort_values(["f1_raw"], ascending=False)
x

,experiment,f1_raw,cm
2,cutoffsepsis-3_no-prediction_24-12_240-240,0.875187,[8278 480 674 284]
3,cutoffsepsis-3_no-prediction_24-12_48-72,0.874505,[8335 423 707 251]
10,TH1.0025_sepsis-3_prediction_24-12_48-72,0.873794,[8255 503 672 286]
8,TH1.0025_sepsis-3_prediction_24-12_24-96,0.873677,[8295 463 693 265]
9,TH1.0025_sepsis-3_prediction_24-12_240-240,0.873287,[8189 569 641 317]
1,cutoffsepsis-3_no-prediction_24-12_24-96,0.873090,[8351 407 724 234]
7,TH1.0025_sepsis-3_prediction_24-12_12-24,0.871553,[8460 298 782 176]
4,cutoffsepsis-3_no-prediction_48-24_168-168,0.871523,[8193 565 656 302]
0,cutoffsepsis-3_no-prediction_24-12_12-24,0.871317,[8476 282 790 168]
5,cutoffsepsis-3_no-prediction_48-24_24-72,0.870441,[8293 465 714 244]


### Weighting True Positives Higher

In [56]:
def compute_weighted_results(path, weight):
    # load results and get predicted arrays

    df = convert_result_to_df(path)
    # get mean t_sepsis to compare methods
    df["t_sepsis"] = df["t_sepsis"].replace({"False":np.nan})
    t_sep, hour_window = compare_t_sepsis(df)

    df["ts_ind"] = df["ts_ind"].astype(int)
    df = df.sort_values(by=["ts_ind"], ascending=True)
    # get indexes of False and True Sepsis labels in results
    noSeps = df.loc[df["Sepsis"] == "False"]
    Seps = df.loc[df["Sepsis"] == "True"]
    neg_hadm_IDs = list(map(int, noSeps["ts_ind"]))
    pos_hadm_IDs = list(map(int, Seps["ts_ind"]))

    # get t of identification of positive labeled patients
    t_ident = get_t_identification(Seps["t_sofa"].astype(float).values, Seps["t_IV"].astype(float).values, Seps["t_cultures"].astype(float).values)
    
    #make predicted df for scoring
    predicted = pd.DataFrame(index=df["ts_ind"], columns=["pred"])
    # 0 for each patient where Sepsis was False
    predicted[predicted.index.isin(neg_hadm_IDs)] = 0
    # 1 for each patient where Sepsis was True
    predicted[predicted.index.isin(pos_hadm_IDs)] = 1
    adj_predicted = predicted.loc[predicted.index.isin(all_possible_)]

    

    # precision, recall f1 (fbeta=1.0) on raw data
    precision, recall, f1_score, support = score(y_true=ground_truth["sepsis_label"].values.astype(int), y_pred=predicted["pred"].values.astype(int), beta=weight, average="weighted")
    auroc_score = auroc(y_true=ground_truth["sepsis_label"].values.astype(int), y_score=predicted["pred"].values.astype(int), average="weighted")
    cm=confusion_matrix(ground_truth["sepsis_label"].values.astype(int), predicted["pred"].values.astype(int)).ravel()
    #print(report(y_true=ground_truth.values.astype(int), y_pred=predicted["pred"].values.astype(int)))

    # precision, recall f1 (fbeta=1.0) on patients that include IV and cultures
    precision_adj, recall_adj, f1_score_adj, support_adj = score(y_true=adj_ground_truth["sepsis_label"].values.astype(int), y_pred=adj_predicted["pred"].values.astype(int), beta=weight,average="weighted")
    auroc_score_adj = auroc(y_true=adj_ground_truth["sepsis_label"].values.astype(int), y_score=adj_predicted["pred"].values.astype(int), average="weighted")
    cm_adj=confusion_matrix(adj_ground_truth["sepsis_label"].values.astype(int), adj_predicted["pred"].values.astype(int)).ravel()

    #print(report(y_true=adj_ground_truth.values.astype(int), y_pred=adj_predicted["pred"].values.astype(int)))

    return  t_sep, hour_window, t_ident, auroc_score, auroc_score_adj, precision, recall, f1_score, support, precision_adj, recall_adj, f1_score_adj, support_adj, cm, cm_adj

In [ ]:
weight = 0.5 #beta multiplier for precision_recall_fscore_support: 1.0 = F1, > 1 favors recall, < 1 favors precision.
results_path = "./results/forecast/test/compare/"
directory = os.fsencode(results_path)
for file in os.listdir(directory):
    name = file.decode('UTF-8')
    t_sepsis, hour_window, t_ident, auroc_score, auroc_score_adj, precision, recall, f1_score, support, precision_adj, recall_adj, f1_score_adj, support_adj, cm, cm_adj = compute_weighted_results(os.path.join(directory, file), weight)
    new_row = {"experiment":name[:-4],"t_sepsis_mean":t_sepsis, "24_hour_window":hour_window, "t_ident":t_ident, "AUROC":auroc_score,"AUROC_adj":auroc_score_adj,"precision_raw":precision, "precision_adj":precision_adj, "recall_raw":recall, "recall_adj":recall_adj, "f1_raw":f1_score, "f1_adj":f1_score_adj, "support":support, "support_adj":support_adj, "cm": cm, "cm_adj":cm_adj}
    df2 = df.append(new_row, ignore_index=True)
    df2.to_csv('./results/forecast/weighted_full_forecast_report.csv', mode='a', index=False, header=False)

In [61]:
# tn, fp, fn, tp
col = ["experiment", "t_sepsis_mean", "24_hour_window", "t_ident","AUROC", "AUROC_adj","precision_raw", "precision_adj", "recall_raw", "recall_adj", "f1_raw", "f1_adj", "support", "support_adj", "cm", "cm_adj"]
results = pd.read_csv("./results/forecast/weighted_full_forecast_report.csv", names=col, header=None)
w = results[["experiment", "f1_raw", "precision_raw", "recall_raw","cm"]].sort_values(["f1_raw"], ascending=False)
w

,experiment,f1_raw,precision_raw,recall_raw,cm
9,TH1.0025_sepsis-3_prediction_24-12_240-240,0.872043,0.871242,0.875463,[8189 569 641 317]
2,cutoffsepsis-3_no-prediction_24-12_240-240,0.872042,0.870185,0.881227,[8278 480 674 284]
10,TH1.0025_sepsis-3_prediction_24-12_48-72,0.870986,0.869286,0.879065,[8255 503 672 286]
3,cutoffsepsis-3_no-prediction_24-12_48-72,0.870037,0.867638,0.883697,[8335 423 707 251]
8,TH1.0025_sepsis-3_prediction_24-12_24-96,0.869937,0.867791,0.881021,[8295 463 693 265]
4,cutoffsepsis-3_no-prediction_48-24_168-168,0.869936,0.868922,0.874331,[8193 565 656 302]
11,TH1.0025_sepsis-3_prediction_48-24_168-168,0.869377,0.870125,0.866509,[8080 678 619 339]
1,cutoffsepsis-3_no-prediction_24-12_24-96,0.868077,0.865481,0.883594,[8351 407 724 234]
12,TH1.0025_sepsis-3_prediction_48-24_24-72,0.867315,0.865829,0.874022,[8214 544 680 278]
5,cutoffsepsis-3_no-prediction_48-24_24-72,0.866265,0.863877,0.878654,[8293 465 714 244]
